In [93]:
import os
import sys

sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib as plt
import sklearn
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from numpy import zeros
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
le = LabelEncoder()

df = pd.read_hdf("../data_extract/features_dataset_2b.h5")
df['fid'] = le.fit_transform(df['fname'])

In [94]:
chlist = ["chroma{}".format(i) for i in range(12)]
log_chroma_list = ["lc{}".format(i) for i in range(12)]
df[log_chroma_list] = np.log(df[chlist])
lclist = ["mfcc{}".format(i) for i in range(14)] + log_chroma_list

In [95]:
NUM_FILES = max(df['fid'])+1
ndf_features = list()
ndf_labels = list()
for i in range(NUM_FILES):
    ndf_features.append(df[lclist].loc[df['fid']==i])
    ndf_labels.append(df[['id']].loc[df['fid']==i])

In [ ]:
#ndf_labels[1100]

In [ ]:
f_train, f_test, _, _ = train_test_split(range(NUM_FILES), range(NUM_FILES), test_size=0.3, random_state=42)

features_train = ndf_features[f_train[0]]
labels_train = ndf_labels[f_train[0]]
for i in range(1,len(f_train)-1):
    features_train = features_train.append(ndf_features[f_train[i]])
    labels_train = labels_train.append(ndf_labels[f_train[i]])

scaler = preprocessing.StandardScaler().fit(features_train)
features_train = scaler.transform(features_train)

In [85]:
#len(f_train)
#labels_train.shape

In [90]:
def train_SVM(x_train,y_train):
    svm_clf = OneVsRestClassifier(LinearSVC(C=1., random_state=42))
    svm_model = svm_clf.fit(x_train, y_train)
    return svm_model

In [53]:
model = train_SVM(features_train,labels_train)

In [91]:
def predict(model,file_id):
    x_test = ndf_features[file_id]
    x_test = scaler.transform(x_test)
    output = model.predict(x_test)
    return np.argmax(np.bincount(output))

def label(file_id):
    l = ndf_labels[file_id]['id'].tolist()
    return l[0]

In [82]:
output = [predict(model,i) for i in f_test]
labels_test = [label(i) for i in f_test]

In [92]:
print 100*accuracy_score(output,labels_test)

56.3501849568
